## 

A prototype

#### Objective

The objective is to test whether 

Categorization of relations 
Changing the value of an attribute (e.g. onStatus) of one device/thing (e.g. smart-socket-1) may affect attributes of another device (e.g. smart-lamp-2)

We wanna make sure this doesn’t happen/users should get warned for altering a thing that has other things dependent on that 

```Cypher
MATCH (a:PowerSource {id: ‘smart-socket-1’})-[r:Powers]->(b:Appliance)
SET b.onStatus = False
RETURN b
```

```Cypher
MATCH (a:PowerSource {id: ‘smart-socket-1’})-[r:Powers]->(b:Appliance)
<detecting mechanism>
SET b.onStatus = False
RETURN b
```

#### Method

In this prototype, I will be using Neo4j's built-in example database, `Movie DBMS`.

One existing relationships in this db model could help with our prototyping process `Acted_in`

implies a dependency of an actor on a movie
analogous to the power source appliance relationship 

`onStatus`

##### Use cases to build model after

* power(on), appliance(off) --> wanting to switch off power

* power(off), appliance(on) --> wanting to switch on appliance

#### Steps
1. Modidying test data
* Add attributes to data
2. Building out the feature
* Write Cypher query that affects power/host's attributes
* Convert Cypher query into python function 
* Control statement to check power/appliance, determine whether to raise an alert

### Set up and test the connection

In [1]:
from py2neo import Graph

In [2]:
remote_url =  "neo4j://localhost:7687"
remote_user = "kaiser_zzk"
remote_pw =   "123465789"

In [3]:
def t1():
    graph = Graph(remote_url, auth=(remote_user, remote_pw))
    q = "match (r:Person) where r.name='Tom Hanks' return r"
    res = graph.run(q)

    for r in res:
        print(r)

In [4]:
t1()

Node('Person', born=1956, name='Tom Hanks')


### Modify `Movie` Nodes

Create and add an attribute that only accepts `True` or `False` as its values 

In [51]:
graph = Graph(remote_url, auth=(remote_user, remote_pw))

In [23]:
createPropertyTestStr = "MATCH (n:Movie) SET n.isTest2 = True"

In [24]:
def createPropertyTest():
    graph.run(createPropertyTestStr)

In [25]:
createPropertyTest()

There is currently no built-in way to enforce a range of values for a property in Neo4j (?)

In [37]:
def checkNodePropertyValue():
    q = "MATCH (n:Movie) WHERE n.title='When Harry Met Sally' RETURN n.released"
    num = graph.run(q)
    
    return num

In [52]:
checkNodePropertyValue()

n.released
1998


checking the value(s) of the related node(s) of the "root" node

In [54]:
def checkRelatedNodesPropertyValue():
    q = "MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'When Harry Met Sally' RETURN p.born"
    years = graph.run(q)
    
    for y in years:
        print(y)

In [53]:
checkRelatedNodesPropertyValue()

1956
1948
1949
1961


In [59]:
graph.run("MATCH (n:Person) SET n.isTest3 = True")

(No data)

##### So far, we have modeled all the features we want to have for prototyping

### Modeling the process

When modifying a depender node, a Cypher query (or a function that invokes the query) attempts to alter the value of status property of the node 

In [62]:
alterStr = "MATCH (m:Movie) WHERE m.title='When Harry Met Sally' SET m.isTest2 = True"

For our propose, we want run examination on the dependee `Movie` node with title "When Harry Met Sally", i.e. iterate through all its depender `Person` nodes, **before** it is being modified (value of `isTest2` set to `True`) 

In [ ]:
"MATCH (m:Movie {title: 'When Harry Met Sally'}) \
CALL { \
    MATCH (p:Person)-[:ACTED_IN]->m \
    RETURN p.born \
}
"

In [ ]:
MATCH (m:Movie) WHERE m.title='When Harry Met Sally'
SET m.isTest2 = 
CASE 
  WHEN <condition> THEN True
  ELSE False
END


In [63]:
testStr = "MATCH (m:Movie)-[:ACTED_IN]->(p:Person) \
WHERE m.title='When Harry Met Sally' \
WITH m, count(p) as people \
WHERE NOT any(p in collect(p) WHERE p.isTest = True) \
SET m.isTest2 = \
CASE \
  WHEN people = 0 THEN False \
  ELSE True \
END"

In [64]:
graph.run(testStr)

ClientError: [Statement.SyntaxError] Invalid use of aggregating function collect(...) in this context (line 1, column 124 (offset: 123))
"MATCH (m:Movie)-[:ACTED_IN]->(p:Person) WHERE m.title='When Harry Met Sally' WITH m, count(p) as people WHERE NOT any(p in collect(p) WHERE p.isTest = True) SET m.isTest2 = CASE   WHEN people = 0 THEN False   ELSE True END"
                                                                                                                            ^

In [111]:
testStr2 = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie) 
WHERE m.title='When Harry Met Sally' 
WITH m, collect(p.isTest2) as peopleTests 
SET m.isTest2 =  
CASE  
  WHEN any(test in peopleTests WHERE test = True) THEN False 
  ELSE True 
END 
RETURN 
CASE 
  WHEN any(test in peopleTests WHERE test = True) THEN 'Cannot switch off' 
  ELSE 'Switched off' 
END AS message
"""

In [112]:
msg = graph.run(testStr2).data()

In [113]:
print(msg)

[{'message': 'Cannot switch off'}]


In [106]:
testStr3 = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title='When Harry Met Sally'
WITH p, collect(p) as people, collect(p.isTest2) as peopleTests
SET p.isTest2 = 
CASE 
  WHEN any(test in peopleTests WHERE test = True) THEN False
  ELSE True
END
RETURN 
CASE 
  WHEN any(test in peopleTests WHERE test = True) THEN 'The condition is not met'
  ELSE 'The condition is met'
END AS message,
[p in people WHERE p.isTest2=True] AS peopleNotMeetingCondition
"""

In [123]:
testStr4  = """
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title='When Harry Met Sally'
WITH collect(p) as people, collect(p.isTest2) as peopleTests
RETURN {
  message: 
    CASE 
      WHEN any(test in peopleTests WHERE test = True) THEN 'Node is still ON'
      ELSE 'All clear; proceed to switch OFF power'
    END,
  nodesNotMeetingCondition: [p in people WHERE p.isTest2=True]
} AS result

"""

In [124]:
msg = graph.run(testStr4).data()

In [125]:
print(msg)

[{'result': {'nodesNotMeetingCondition': [Node('Person', born=1949, isTest2=True, isTest3=True, name='Bruno Kirby')], 'message': 'Node is still ON'}}]


### finallll

In [57]:
# Set up connection to Neo4j db

remote_url =  "neo4j://localhost:7687"
remote_user = "kaiser_zzk"
remote_pw =   "123465789"

In [58]:
graph = Graph(remote_url, auth=(remote_user, remote_pw))